In [3]:
import netCDF4
import json
import geojson
import numpy as np
import pandas

In [4]:
filename = "data/TRANSPORT_coast_wadden.nc"

In [5]:
ds = netCDF4.Dataset(filename)

In [28]:
angle = ds.variables['cstangle'][:]
print(angle)
neg = (ds.variables['QSnet'][:] < 0) * 180
print(neg)
angle += neg
print(angle)

[239.74 262.07 284.24 287.9 303.69 300.77 312.71 327.53 322.97 320.57
 316.67 324.74 336.67 18.43 309.35 327.22 345.58 341.05 341.56 342.73
 352.12 21.48 322.37 355.08 359.46 358.99 0.0 352.07 353.95 1.61 350.83
 326.86 354.88 352.0 345.21 344.43]
[180 180 180 180   0   0   0   0   0   0   0   0   0   0 180   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0 180   0   0   0   0]
[419.74 442.07 464.24 467.9 303.69 300.77 312.71 327.53 322.97 320.57
 316.67 324.74 336.67 18.43 489.35 327.22 345.58 341.05 341.56 342.73
 352.12 21.48 322.37 355.08 359.46 358.99 0.0 352.07 353.95 1.61 350.83
 506.86 354.88 352.0 345.21 344.43]


In [20]:
ds.variables['time'][0]

masked_array(data=1502755200.0,
             mask=False,
       fill_value=9.969209968386869e+36)

In [135]:
netCDF4.num2date(ds.variables['time'][0], 'seconds since 1970-01-01 00:00:00 0:00')

datetime.datetime(2017, 8, 15, 0, 0)

In [21]:
ds = netCDF4.Dataset(filename)

def nc2feature(ind):
    variables = {
        'lat': {"var": 'lat', "slice": np.s_[ind]},
        'lon': {"var": 'lon', "slice": np.s_[ind]},
        'index': {"var": 'QSnet', "slice": np.s_[ind]},
    }
    data = {}
    for var, props in variables.items():
        data[var] = ds.variables[props['var']][props['slice']]
        
    angle = ds.variables['cstangle'][ind]
    neg = (ds.variables['QSnet'][ind] < 0) * 180
    angle += neg
    data['angle'] = angle
    
    coords = [data['lon'].tolist(), data['lat'].tolist()]
    geometry = geojson.Point(coordinates=coords)
    date = netCDF4.num2date(ds.variables['time'][ind], 'seconds since 1970-01-01 00:00:00 0:00')
    properties = {
        "date": date.isoformat(),
        "year": date.year, 
        "angle": data['angle'].tolist(),
        "index": data['index'].tolist()
    }
    return geojson.Feature(id="point_" + str(ind), geometry=geometry, properties=properties)

In [22]:
import itertools
import more_itertools
import tqdm

features = []
for ind in tqdm.tqdm_notebook(range(ds.dimensions['obs'].size)):
    features.append(nc2feature(ind))

In [23]:
collection = geojson.FeatureCollection(features)
with open('testdata.json', 'w') as f:
    geojson.dump(collection, f)